In [10]:
from PIL import Image
import cv2

In [5]:
def change_image_size(input_path, output_path, new_size):
    try:
        # Open the image
        with Image.open(input_path) as img:
            # Resize the image
            resized_img = img.resize(new_size)
            
            # Save the resized image
            resized_img.save(output_path)
            
            print(f"Image reshaped and saved to {output_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [9]:
input_image = "../test_image1.jpg"
output_image = "../RestructureImage/output.jpg"
new_size = (320, 240)  # New width and height

change_image_size(input_image, output_image, new_size)

Image reshaped and saved to ../RestructureImage/output.jpg


In [12]:
image = cv2.imread('path/to/your/image.jpg')
image.shape

[ WARN:0@79.736] global loadsave.cpp:241 findDecoder imread_('path/to/your/image.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'